In [6]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Preproccesing and recostructing a dataset from DLS
Few lines intro

**Learning objectives:**
1. Construct and use CIL processors Centre of rotation and Resizer to preprocess the data
2. Define the projector
3. Apply different reconstruction techniques to a real dataset

In [42]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from ccpi.framework import ImageData, ImageGeometry
from ccpi.framework import AcquisitionGeometry, AcquisitionData

from ccpi.optimisation.algorithms import CGLS, SIRT
from ccpi.optimisation.functions import Norm2Sq, L1Norm
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity
from ccpi.framework import BlockDataContainer

from ccpi.processors import Resizer, CenterOfRotationFinder

from ccpi.io import NEXUSDataReader
from ccpi.astra.operators import AstraProjectorSimple , AstraProjector3DSimple
from ccpi.astra.processors import FBP

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import scipy
from utilities import islicer
from utilities import plotter2D


### Read in the dataset

About the data?

In [8]:
## Set up a reader object pointing to the Nexus data set. Revise path as needed.
# The data is already  corrected for by flat and dark field.
path = os.path.join(sys.prefix, 'share','ccpi','24737_fd_normalised.nxs')
myreader = NEXUSDataReader(nexus_file=path)
data_raw = myreader.load_data()

In [9]:
#Look at the data set
print(type(data_raw))
print(data_raw)
islicer(data_raw, direction=0)

<class 'ccpi.framework.framework.AcquisitionData'>
Number of dimensions: 3
Shape: (91, 135, 160)
Axis labels: {0: 'angle', 1: 'vertical', 2: 'horizontal'}



interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=90), Output()), _dom_cl…

### Use Resizer() to pre-proccess the data

We want to remove the top row data with the dark pixel

Use the processor Resizer()


In [10]:
start_row = 1
end_row = data_raw.shape[1]
roi_crop = [-1,(start_row, end_row),-1]

In [11]:
resizer = Resizer(roi=roi_crop)
resizer.set_input(data_raw)
data_cropped = resizer.get_output()

In [12]:
#Look at the data set
print(type(data_cropped))
print(data_cropped)
islicer(data_cropped, direction=0)

<class 'ccpi.framework.framework.AcquisitionData'>
Number of dimensions: 3
Shape: (91, 134, 160)
Axis labels: {0: 'angle', 1: 'vertical', 2: 'horizontal'}



interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=90), Output()), _dom_cl…

### Use CenterOfRotationFinder()

In [13]:
# Set up CenterOfRotationFinder object to center data.
# Set the output of the normaliser as the input and execute to determine center.
cor = CenterOfRotationFinder()
cor.set_input(data_cropped)
center_of_rotation = cor.get_output()
print( "centre of rotation at pixel: ", center_of_rotation)

centre of rotation at pixel:  86.25


<span style="color:red;font-size:larger">**Exercise 1:**</span> Use Resizer to crop the pojections and correct the Centre of rotation offset.

In [15]:
#get image width
num_pixels_y = data_cropped.shape[2]
cor_shift = center_of_rotation - num_pixels_y/2

In [16]:
start_column = int(round(2.*cor_shift))
end_column = data_cropped.shape[2]

roi_crop = [-1,-1,(start_column, end_column)]
resizer = Resizer(roi=roi_crop)
resizer.set_input(data_cropped)
data_centred = resizer.get_output()

In [17]:
#Look at the data set
print(type(data_centred))
print(data_centred)
islicer(data_centred, direction=0)

<class 'ccpi.framework.framework.AcquisitionData'>
Number of dimensions: 3
Shape: (91, 134, 148)
Axis labels: {0: 'angle', 1: 'vertical', 2: 'horizontal'}



interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=90), Output()), _dom_cl…

### Set up the data ready for ASTRA

ASTRA expects the data in the order `['vertical','angle','horizontal']` so we need to permute the dataset

In [18]:
print(data_centred)

Number of dimensions: 3
Shape: (91, 134, 148)
Axis labels: {0: 'angle', 1: 'vertical', 2: 'horizontal'}



In [19]:
data = data_centred.subset(dimensions=['vertical','angle','horizontal'])

In [20]:
print(data)

Number of dimensions: 3
Shape: (134, 91, 148)
Axis labels: {0: 'vertical', 1: 'angle', 2: 'horizontal'}



ASTRA takes the angles as radians so let's convert the angles geometry data

In [21]:
#convert the angles to radians
if(data.geometry.angle_unit == 'degree'):
    data.geometry.angle_unit = 'radian'
    data.geometry.angles = data.geometry.angles * np.pi /180.

And finally convert from intensity to attenuation data

In [22]:
#Convert from intensity to attenuation data
#how can I make sure thy don't execute this more than once?
data.fill(-np.log(data.as_array()))
#Look at the data set
print(type(data))
print(data)
islicer(data, direction=1) #remember we've transposed the data so angles is now the 2nd axis

<class 'ccpi.framework.framework.AcquisitionData'>
Number of dimensions: 3
Shape: (134, 91, 148)
Axis labels: {0: 'vertical', 1: 'angle', 2: 'horizontal'}



interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=90), Output()), _dom_cl…

### Define the Geometry

#### Acquistion geometry
In the 2D example we used:<br>
`ag = AcquisitionGeometry(geom_type='parallel', dimension='2D', angles=angles, pixel_num_h=number_pixels_x)`<br>

For 3D we need to change the dimension description to ` dimension='3D'`, and pass the number of vertical pixels as `pixel_num_v`<br>

However we've been using the acquistion geometry throughout this notebook so let's just clone the version we've already set up.

#### Image geometry
In the 2D example we used:<br>
`ig = ImageGeometry(voxel_num_x = num_voxels_xy, voxel_num_y = num_voxels_xy)`

For ad 3D reconstruction we also need to pass the number of voxels we want in the $z$-direction as `voxel_num_z`

If you create the image geometry from the acquisiton geometry this is set to `pixel_num_h` by default.

In [23]:
# Create Acquisition Geometry
ag = data.geometry.clone()

# Create Image Geometry
ig = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                   voxel_num_y=ag.pixel_num_h, 
                   voxel_num_z=ag.pixel_num_v)

## FBP Reconstruction

Reconstruct the data set using the FBP processor from ASTRA

`from ccpi.astra.processors import FBP`

We Run this in the same way as the processors introduced above.

In [43]:
#Initialise the processor
fbp = FBP(ig, ag, device='gpu')

In [44]:
#set the input
fbp.set_input(data)

In [45]:
#Run the procesor
FBP_output = fbp.get_output()

In [46]:
#plot the results
islicer(FBP_output, direction=0,fix_range=False)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

### Define the projector

In the 2D example we used the 2D projector from ASTRA<br>
`'AstraProjectorSimple(volume_geometry, sinogram_geometry, device)`

Use ASTRA's 3D projector, note this projector is GPU only<br>
`AstraProjector3DSimple(volume_geometry, sinogram_geometry)`

In [24]:
# Define the projector object
print ("Define projector")
Cop = AstraProjector3DSimple(ig, ag)

Define projector


### Run SIRT

In [25]:
#setup SIRT
x_init = ig.allocate(0)
sirt = SIRT(x_init=x_init, operator=Cop, data=data, update_objective_interval = 10)
sirt.max_iteration = 1000

SIRT setting up
SIRT configured


In [26]:
#run the algorithm
sirt.run(100, verbose = True)

     Iter   Max Iter     Time/Iter            Objective
                               [s]                     
        0       1000         0.000          1.01151e+06
       10       1000         0.048          1.01151e+06
       20       1000         0.046          1.88900e+04
       30       1000         0.047          6.70392e+03
       40       1000         0.047          3.32638e+03
       50       1000         0.046          2.01420e+03
       60       1000         0.046          1.38821e+03
       70       1000         0.047          1.03907e+03
       80       1000         0.047          8.20673e+02
       90       1000         0.047          6.72625e+02
      100       1000         0.047          5.66454e+02


In [27]:
#plot the results
SIRT_output = sirt.get_output()
islicer(SIRT_output, direction=0,fix_range=False)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

### Run Tikhonov CGLS

In [28]:
#define the operator
alpha = 5
L = Gradient(ig)
operator_block = BlockOperator( Cop, alpha * L, shape=(2,1))

In [29]:
#define the data b
data_block = BlockDataContainer(data, L.range_geometry().allocate(0))

In [30]:
#setup Tikonov
L = Gradient(ig)
alpha = 75

x_init = ig.allocate(0)
cgls_tikhonov = CGLS(x_init=x_init, operator=operator_block, data=data_block, update_objective_interval = 10)
cgls_tikhonov.max_iteration = 1000

CGLS setting up
CGLS configured


In [31]:
#run the algorithm
cgls_tikhonov.run(100, verbose = True)

     Iter   Max Iter     Time/Iter            Objective
                               [s]                     
        0       1000         0.000          1.01151e+06
       10       1000         0.119          1.25432e+05
       20       1000         0.117          9.16034e+02
       30       1000         0.115          7.70432e+02
       40       1000         0.117          7.56054e+02
       50       1000         0.115          7.52701e+02
       60       1000         0.115          7.51617e+02
       70       1000         0.116          7.51192e+02
       80       1000         0.116          7.51052e+02
       90       1000         0.117          7.51011e+02
      100       1000         0.116          7.50997e+02


In [32]:
#plot the results
CGLS_tikhonov_output = cgls_tikhonov.get_output()

islicer(CGLS_tikhonov_output, direction=0,fix_range=False)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

### Summary

In [37]:
#compare the outputs of unregularise and regularised CGLS

islicer(SIRT_output, direction=0,fix_range=False)
islicer(CGLS_tikhonov_output, direction=0,fix_range=False)
islicer(FBP_output, direction=0,fix_range=False)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…